In [6]:
import numpy as np
import plotly.graph_objects as go

def groove_parametrization_function(alpha_HG, beta_HG, hg_HG, D, LoD, gamma):
    # Calculate dependent variables
    L = LoD * D
    L_g = gamma * L
    L_land = L - L_g

    # Parameters
    N_HG = 28  # number of grooves
    beta_HG = beta_HG * np.pi / 180

    # Calculation for CAD
    Spiral_step = np.pi * D * np.tan(beta_HG)
    a_HG = (np.pi * D * alpha_HG) / N_HG

    # Example geometry for V grooves
    beta = beta_HG
    theta = np.pi / 2 - beta
    D_g = L_g * np.tan(theta)
    a_g = a_HG

    # Define parallelogram vertices
    v1 = np.array([0, L_land, 0])
    v2 = np.array([-D_g, L_g + L_land, 0])
    v4 = np.array([-a_g, L_land, 0])

    # Define bending parameters
    xmin = -Spiral_step / 2
    xmax = Spiral_step / 2
    Rc = D / 2
    zmaxc = L
    zminc = -L
    N_mesh = 50 * int(np.ceil(Rc / 5))

    # Create parallelogram surface
    X, Y = np.meshgrid(np.linspace(0, 1, N_mesh), np.linspace(0, 1, N_mesh))
    P = np.tile(v1, (X.size, 1)) + np.outer(X.flatten(), v2 - v1) + np.outer(Y.flatten(), v4 - v1)
    P = P.reshape(X.shape[0], X.shape[1], 3)

    # Do the bending
    Z2 = P[:, :, 1]
    X1 = P[:, :, 0]
    phi = (X1 - xmin) / (xmax - xmin) * 2 * np.pi
    X2 = Rc * np.cos(phi)
    Y2 = Rc * np.sin(phi)

    # Plot cylinder
    u = np.linspace(0, 2 * np.pi, 100)
    v = np.linspace(zminc, zmaxc, 100)
    x = Rc * np.outer(np.cos(u), np.ones_like(v))
    y = Rc * np.outer(np.sin(u), np.ones_like(v))
    z = np.outer(np.ones_like(u), v)

    

    # Plot bent parallelogram
    P2 = np.dstack((X2, Y2, Z2))

    # Extract the edge points
    edge1 = P2[0, :, :]  # First row
    edge2 = P2[-1, :, :]  # Last row
    edge3 = P2[:, 0, :]  # First column
    edge4 = P2[:, -1, :]  # Last column

    # Concatenate all edge points into one matrix for easy plotting
    allEdgePoints = np.vstack((edge1, edge4, edge2[::-1], edge3[::-1]))
    #allEdgePoints = np.vstack((edge1, edge2, edge3, edge4))

    # Scale parallelogram to new radius
    Rc_new = Rc - hg_HG
    scale_factor = Rc_new / Rc
    P2_new = P2.copy()
    P2_new[:, :, 0] = P2_new[:, :, 0] * scale_factor
    P2_new[:, :, 1] = P2_new[:, :, 1] * scale_factor

    # Extract the edge points
    edge1_new = P2_new[0, :, :]  # First row
    edge2_new = P2_new[-1, :, :]  # Last row
    edge3_new = P2_new[:, 0, :]  # First column
    edge4_new = P2_new[:, -1, :]  # Last column

    # Concatenate all edge points into one matrix for easy plotting
    allEdgePoints_new = np.vstack((edge1_new, edge4_new, edge2_new[::-1], edge3_new[::-1]))
    #allEdgePoints_new = np.vstack((edge1_new, edge2_new, edge3_new, edge4_new))

    # Plot the edge points
    ##fig.add_trace(go.Scatter3d(x=allEdgePoints[:, 0], y=allEdgePoints[:, 1], z=allEdgePoints[:, 2], mode='markers', marker=dict(size=2, color='red')))

    # Plot the edge points of bottom of groove
    ##fig.add_trace(go.Scatter3d(x=allEdgePoints_new[:, 0], y=allEdgePoints_new[:, 1], z=allEdgePoints_new[:, 2], mode='markers', marker=dict(size=2, color='red')))

    fig = go.Figure(data=[go.Surface(x=x, y=y, z=z)])
    
    # Plot the edge points as a line (to check continuous loop: verified, it works)
    fig.add_trace(go.Scatter3d(x=allEdgePoints[:, 0], y=allEdgePoints[:, 1], z=allEdgePoints[:, 2], mode='lines', line=dict(color='blue', width=2)))

    # Plot the edge points of bottom of groove (to check continuous loop: verified it works)
    fig.add_trace(go.Scatter3d(x=allEdgePoints_new[:, 0], y=allEdgePoints_new[:, 1], z=allEdgePoints_new[:, 2], mode='lines', line=dict(color='red', width=2)))


    fig.show()

    return allEdgePoints, allEdgePoints_new

In [7]:
alpha_HG = 0.68
beta_HG = 135
D = 16
LoD = 1.7
gamma = 0.75
hg_HG = 1
# Call the function with your parameters here
allEdgePoints, allEdgePoints_new = groove_parametrization_function(alpha_HG, beta_HG, hg_HG, D, LoD, gamma)

In [3]:
allEdgePoints

array([[-8.00000000e+00,  9.79717439e-16,  6.80000000e+00],
       [-7.99734634e+00,  2.06037822e-01,  7.00606061e+00],
       [-7.98938710e+00,  4.11938954e-01,  7.21212121e+00],
       ...,
       [-7.99996199e+00, -2.46612366e-02,  6.80000000e+00],
       [-7.99999050e+00, -1.23306329e-02,  6.80000000e+00],
       [-8.00000000e+00,  9.79717439e-16,  6.80000000e+00]])